In [1]:
from __future__ import print_function, division

from keras.datasets import cifar10
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply, concatenate
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D, Lambda
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.utils import to_categorical
import keras.backend as K
from PIL import Image
import matplotlib.pyplot as plt
import os
import numpy as np

class INFOGAN():
    def __init__(self):
        self.img_rows = 32
        self.img_cols = 32
        self.channels = 3
        self.num_classes = 10
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 72


        optimizer = Adam(0.0002, 0.5)
        losses = ['binary_crossentropy', self.mutual_info_loss]

        # Build and the discriminator and recognition network
        self.discriminator, self.auxilliary = self.build_disk_and_q_net()

        self.discriminator.compile(loss=['binary_crossentropy'],
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build and compile the recognition network Q
        self.auxilliary.compile(loss=[self.mutual_info_loss],
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise and the target label as input
        # and generates the corresponding digit of that label
        gen_input = Input(shape=(self.latent_dim,))
        img = self.generator(gen_input)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated image as input and determines validity
        valid = self.discriminator(img)
        # The recognition network produces the label
        target_label = self.auxilliary(img)

        # The combined model  (stacked generator and discriminator)
        self.combined = Model(gen_input, [valid, target_label])
        self.combined.compile(loss=losses,
            optimizer=optimizer)


    def build_generator(self):

        model = Sequential()

        model.add(Dense(128 * 8 * 8, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((8, 8, 128)))
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(self.channels, kernel_size=3, padding='same'))
        model.add(Activation("tanh"))

        gen_input = Input(shape=(self.latent_dim,))
        img = model(gen_input)

        model.summary()

        return Model(gen_input, img)


    def build_disk_and_q_net(self):

        img = Input(shape=self.img_shape)

        # Shared layers between discriminator and recognition network
        model = Sequential()
        model.add(Conv2D(64, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        # model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(256, kernel_size=3, strides=2, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(512, kernel_size=3, strides=2, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Flatten())

        img_embedding = model(img)

        # Discriminator
        validity = Dense(1, activation='sigmoid')(img_embedding)

        # Recognition
        q_net = Dense(128, activation='relu')(img_embedding)
        label = Dense(self.num_classes, activation='softmax')(q_net)

        # Return discriminator and recognition network
        return Model(img, validity), Model(img, label)


    def mutual_info_loss(self, c, c_given_x):
        """The mutual information metric we aim to minimize"""
        eps = 1e-8
        conditional_entropy = K.mean(- K.sum(K.log(c_given_x + eps) * c, axis=1))
        entropy = K.mean(- K.sum(K.log(c + eps) * c, axis=1))

        return conditional_entropy + entropy

    def sample_generator_input(self, batch_size):
        # Generator inputs
        sampled_noise = np.random.normal(0, 1, (batch_size, 62))
        sampled_labels = np.random.randint(0, self.num_classes, batch_size).reshape(-1, 1)
        sampled_labels = to_categorical(sampled_labels, num_classes=self.num_classes)

        return sampled_noise, sampled_labels

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, y_train), (_, _) = cifar10.load_data()

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        # X_train = np.expand_dims(X_train, axis=3)
        y_train = y_train.reshape(-1, 1)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        nb_batches = int(X_train.shape[0] / batch_size)
        global_step = 0

        for epoch in range(epochs):

            for index in range(nb_batches):
                global_step += 1
                imgs = X_train[index * batch_size:(index + 1) * batch_size]
                # Sample noise and categorical labels
                sampled_noise, sampled_labels = self.sample_generator_input(batch_size)
                gen_input = np.concatenate((sampled_noise, sampled_labels), axis=1)

                # Generate a half batch of new images
                gen_imgs = self.generator.predict(gen_input)

                # Train on real and generated data
                d_loss_real = self.discriminator.train_on_batch(imgs, valid)
                d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)

                # Avg. loss
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

                # ---------------------
                #  Train Generator and Q-network
                # ---------------------

                g_loss = self.combined.train_on_batch(gen_input, [valid, sampled_labels])

                # Plot the progress
                print("epoch:%d step:%d [D loss: %.2f, acc.: %.2f%%] [Q loss: %.2f] [G loss: %.2f]" % (
                    epoch,global_step, d_loss[0], 100 * d_loss[1], g_loss[1], g_loss[2]))

                # If at save interval => save generated image samples
                if global_step % sample_interval == 0:
                    self.sample_images(epoch, global_step)

    def sample_images(self, epoch, global_step):
        r,c=10,10


        noise = np.random.normal(0, 1, (r * c, 62))
        sampled_labels = np.array([num for _ in range(r) for num in range(c)])
        label = to_categorical(sampled_labels, num_classes=self.num_classes)
        gen_input = np.concatenate((noise, label), axis=1)
        generated_images = self.generator.predict(gen_input)

        generated_images = np.asarray((generated_images * 127.5 + 127.5).astype(np.uint8))

        def vis_square(data, padsize=1, padval=0):
            # force the number of filters to be square
            n = int(np.ceil(np.sqrt(data.shape[0])))
            padding = ((0, n ** 2 - data.shape[0]), (0, padsize), (0, padsize)) + ((0, 0),) * (data.ndim - 3)
            data = np.pad(data, padding, mode='constant', constant_values=(padval, padval))

            # tile the filters into an image
            data = data.reshape((n, n) + data.shape[1:]).transpose((0, 2, 1, 3) + tuple(range(4, data.ndim + 1)))
            data = data.reshape((n * data.shape[1], n * data.shape[3]) + data.shape[4:])
            return data

        img = vis_square(generated_images)
        if not os.path.isdir('images_infogan_cifar10'):
            os.mkdir('images_infogan_cifar10')
        Image.fromarray(img).save(
            "images_infogan_cifar10/epoch_%d_step_%d.png" % (epoch, global_step))



if __name__ == '__main__':
    infogan = INFOGAN()
    infogan.train(epochs=50, batch_size=64, sample_interval=200)


Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 8192)              598016    
_________________________________________________________________
reshape_1 (Reshape)          (None, 8, 8, 128)         0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 8, 8, 128)         512       
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 16, 16, 128)       147584    
_________________________________________________________________
activation_1 (Activation)    (None, 16, 16, 128)       0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 16, 16, 128)       512       
__________

/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch:0 step:1 [D loss: 0.96, acc.: 43.75%] [Q loss: 0.77] [G loss: 2.93]
epoch:0 step:2 [D loss: 0.89, acc.: 53.91%] [Q loss: 0.91] [G loss: 3.03]
epoch:0 step:3 [D loss: 0.95, acc.: 51.56%] [Q loss: 0.96] [G loss: 3.16]
epoch:0 step:4 [D loss: 0.82, acc.: 56.25%] [Q loss: 1.13] [G loss: 2.78]
epoch:0 step:5 [D loss: 0.64, acc.: 67.19%] [Q loss: 1.19] [G loss: 2.96]
epoch:0 step:6 [D loss: 0.69, acc.: 60.94%] [Q loss: 1.05] [G loss: 2.63]
epoch:0 step:7 [D loss: 0.84, acc.: 60.94%] [Q loss: 0.99] [G loss: 2.79]
epoch:0 step:8 [D loss: 0.83, acc.: 59.38%] [Q loss: 1.16] [G loss: 2.84]
epoch:0 step:9 [D loss: 0.79, acc.: 53.91%] [Q loss: 0.89] [G loss: 2.73]
epoch:0 step:10 [D loss: 0.68, acc.: 61.72%] [Q loss: 1.01] [G loss: 2.90]
epoch:0 step:11 [D loss: 0.75, acc.: 64.84%] [Q loss: 1.12] [G loss: 2.56]
epoch:0 step:12 [D loss: 0.74, acc.: 62.50%] [Q loss: 1.16] [G loss: 2.50]
epoch:0 step:13 [D loss: 0.69, acc.: 62.50%] [Q loss: 1.18] [G loss: 2.61]
epoch:0 step:14 [D loss: 0.73, acc

epoch:0 step:113 [D loss: 0.59, acc.: 71.88%] [Q loss: 1.39] [G loss: 0.44]
epoch:0 step:114 [D loss: 0.55, acc.: 67.97%] [Q loss: 1.29] [G loss: 0.21]
epoch:0 step:115 [D loss: 0.52, acc.: 72.66%] [Q loss: 1.35] [G loss: 0.40]
epoch:0 step:116 [D loss: 0.69, acc.: 64.06%] [Q loss: 1.35] [G loss: 0.36]
epoch:0 step:117 [D loss: 0.50, acc.: 71.09%] [Q loss: 1.41] [G loss: 0.31]
epoch:0 step:118 [D loss: 0.51, acc.: 73.44%] [Q loss: 1.35] [G loss: 0.34]
epoch:0 step:119 [D loss: 0.55, acc.: 68.75%] [Q loss: 1.29] [G loss: 0.29]
epoch:0 step:120 [D loss: 0.61, acc.: 67.19%] [Q loss: 1.50] [G loss: 0.24]
epoch:0 step:121 [D loss: 0.59, acc.: 70.31%] [Q loss: 1.35] [G loss: 0.28]
epoch:0 step:122 [D loss: 0.54, acc.: 74.22%] [Q loss: 1.35] [G loss: 0.27]
epoch:0 step:123 [D loss: 0.63, acc.: 65.62%] [Q loss: 1.38] [G loss: 0.23]
epoch:0 step:124 [D loss: 0.50, acc.: 71.88%] [Q loss: 1.39] [G loss: 0.28]
epoch:0 step:125 [D loss: 0.61, acc.: 61.72%] [Q loss: 1.11] [G loss: 0.20]
epoch:0 step

...

epoch:49 step:38616 [D loss: 0.73, acc.: 26.56%] [Q loss: 0.74] [G loss: 0.00]
epoch:49 step:38617 [D loss: 0.73, acc.: 27.34%] [Q loss: 0.75] [G loss: 0.00]
epoch:49 step:38618 [D loss: 0.73, acc.: 26.56%] [Q loss: 0.74] [G loss: 0.00]
epoch:49 step:38619 [D loss: 0.73, acc.: 26.56%] [Q loss: 0.73] [G loss: 0.00]
epoch:49 step:38620 [D loss: 0.74, acc.: 23.44%] [Q loss: 0.74] [G loss: 0.12]
epoch:49 step:38621 [D loss: 0.73, acc.: 26.56%] [Q loss: 0.73] [G loss: 0.00]
epoch:49 step:38622 [D loss: 0.73, acc.: 22.66%] [Q loss: 0.75] [G loss: 0.00]
epoch:49 step:38623 [D loss: 0.74, acc.: 21.09%] [Q loss: 0.74] [G loss: 0.00]
epoch:49 step:38624 [D loss: 0.72, acc.: 38.28%] [Q loss: 0.73] [G loss: 0.00]
epoch:49 step:38625 [D loss: 0.73, acc.: 31.25%] [Q loss: 0.72] [G loss: 0.00]
epoch:49 step:38626 [D loss: 0.74, acc.: 30.47%] [Q loss: 0.73] [G loss: 0.00]
epoch:49 step:38627 [D loss: 0.73, acc.: 28.91%] [Q loss: 0.72] [G loss: 0.00]
epoch:49 step:38628 [D loss: 0.73, acc.: 24.22%] [Q 

epoch:49 step:38721 [D loss: 0.73, acc.: 30.47%] [Q loss: 0.74] [G loss: 0.00]
epoch:49 step:38722 [D loss: 0.73, acc.: 35.94%] [Q loss: 0.75] [G loss: 0.00]
epoch:49 step:38723 [D loss: 0.73, acc.: 31.25%] [Q loss: 0.73] [G loss: 0.02]
epoch:49 step:38724 [D loss: 0.73, acc.: 32.03%] [Q loss: 0.75] [G loss: 0.00]
epoch:49 step:38725 [D loss: 0.72, acc.: 36.72%] [Q loss: 0.75] [G loss: 0.00]
epoch:49 step:38726 [D loss: 0.73, acc.: 28.12%] [Q loss: 0.74] [G loss: 0.00]
epoch:49 step:38727 [D loss: 0.73, acc.: 26.56%] [Q loss: 0.74] [G loss: 0.01]
epoch:49 step:38728 [D loss: 0.73, acc.: 27.34%] [Q loss: 0.74] [G loss: 0.02]
epoch:49 step:38729 [D loss: 0.72, acc.: 32.03%] [Q loss: 0.74] [G loss: 0.00]
epoch:49 step:38730 [D loss: 0.73, acc.: 21.88%] [Q loss: 0.75] [G loss: 0.00]
epoch:49 step:38731 [D loss: 0.73, acc.: 29.69%] [Q loss: 0.74] [G loss: 0.29]
epoch:49 step:38732 [D loss: 0.72, acc.: 35.94%] [Q loss: 0.74] [G loss: 0.00]
epoch:49 step:38733 [D loss: 0.73, acc.: 33.59%] [Q 

epoch:49 step:38827 [D loss: 0.74, acc.: 22.66%] [Q loss: 0.75] [G loss: 0.00]
epoch:49 step:38828 [D loss: 0.73, acc.: 28.91%] [Q loss: 0.74] [G loss: 0.00]
epoch:49 step:38829 [D loss: 0.72, acc.: 33.59%] [Q loss: 0.72] [G loss: 0.03]
epoch:49 step:38830 [D loss: 0.73, acc.: 32.03%] [Q loss: 0.73] [G loss: 0.00]
epoch:49 step:38831 [D loss: 0.73, acc.: 32.81%] [Q loss: 0.73] [G loss: 0.00]
epoch:49 step:38832 [D loss: 0.73, acc.: 32.81%] [Q loss: 0.74] [G loss: 0.00]
epoch:49 step:38833 [D loss: 0.72, acc.: 31.25%] [Q loss: 0.75] [G loss: 0.00]
epoch:49 step:38834 [D loss: 0.74, acc.: 31.25%] [Q loss: 0.74] [G loss: 0.00]
epoch:49 step:38835 [D loss: 0.73, acc.: 26.56%] [Q loss: 0.73] [G loss: 0.01]
epoch:49 step:38836 [D loss: 0.73, acc.: 27.34%] [Q loss: 0.74] [G loss: 0.01]
epoch:49 step:38837 [D loss: 0.73, acc.: 30.47%] [Q loss: 0.74] [G loss: 0.00]
epoch:49 step:38838 [D loss: 0.72, acc.: 32.81%] [Q loss: 0.73] [G loss: 0.00]
epoch:49 step:38839 [D loss: 0.72, acc.: 37.50%] [Q 

epoch:49 step:38931 [D loss: 0.73, acc.: 32.03%] [Q loss: 0.73] [G loss: 0.00]
epoch:49 step:38932 [D loss: 0.72, acc.: 28.12%] [Q loss: 0.73] [G loss: 0.14]
epoch:49 step:38933 [D loss: 0.72, acc.: 32.03%] [Q loss: 0.72] [G loss: 0.05]
epoch:49 step:38934 [D loss: 0.74, acc.: 25.78%] [Q loss: 0.75] [G loss: 0.00]
epoch:49 step:38935 [D loss: 0.73, acc.: 32.03%] [Q loss: 0.74] [G loss: 0.03]
epoch:49 step:38936 [D loss: 0.72, acc.: 37.50%] [Q loss: 0.74] [G loss: 0.00]
epoch:49 step:38937 [D loss: 0.72, acc.: 33.59%] [Q loss: 0.74] [G loss: 0.00]
epoch:49 step:38938 [D loss: 0.73, acc.: 28.91%] [Q loss: 0.74] [G loss: 0.00]
epoch:49 step:38939 [D loss: 0.73, acc.: 32.03%] [Q loss: 0.74] [G loss: 0.00]
epoch:49 step:38940 [D loss: 0.73, acc.: 26.56%] [Q loss: 0.73] [G loss: 0.00]
epoch:49 step:38941 [D loss: 0.73, acc.: 34.38%] [Q loss: 0.74] [G loss: 0.00]
epoch:49 step:38942 [D loss: 0.72, acc.: 37.50%] [Q loss: 0.74] [G loss: 0.00]
epoch:49 step:38943 [D loss: 0.72, acc.: 32.03%] [Q 

epoch:49 step:39038 [D loss: 0.73, acc.: 31.25%] [Q loss: 0.74] [G loss: 0.00]
epoch:49 step:39039 [D loss: 0.73, acc.: 35.16%] [Q loss: 0.73] [G loss: 0.00]
epoch:49 step:39040 [D loss: 0.73, acc.: 28.91%] [Q loss: 0.73] [G loss: 0.00]
epoch:49 step:39041 [D loss: 0.73, acc.: 25.00%] [Q loss: 0.74] [G loss: 0.00]
epoch:49 step:39042 [D loss: 0.72, acc.: 33.59%] [Q loss: 0.74] [G loss: 0.00]
epoch:49 step:39043 [D loss: 0.72, acc.: 32.81%] [Q loss: 0.74] [G loss: 0.00]
epoch:49 step:39044 [D loss: 0.72, acc.: 32.81%] [Q loss: 0.72] [G loss: 0.00]
epoch:49 step:39045 [D loss: 0.74, acc.: 28.91%] [Q loss: 0.73] [G loss: 0.00]
epoch:49 step:39046 [D loss: 0.73, acc.: 28.91%] [Q loss: 0.74] [G loss: 0.00]
epoch:49 step:39047 [D loss: 0.73, acc.: 31.25%] [Q loss: 0.73] [G loss: 0.00]
epoch:49 step:39048 [D loss: 0.73, acc.: 32.03%] [Q loss: 0.73] [G loss: 0.00]
epoch:49 step:39049 [D loss: 0.73, acc.: 33.59%] [Q loss: 0.73] [G loss: 0.00]
epoch:49 step:39050 [D loss: 0.73, acc.: 32.03%] [Q 